In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
from glob import glob

from IPython.display import display
import numpy as np
import pandas as pd
from tqdm import tqdm

from common.utils import median_ensemble

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from common.summary_utils import EnsembleStatistics
from common.metrics import smape_1, smape_2, nd, mape

# FRED
from resources.fred.dataset import FredDataset, FredMeta
from resources.fred.evaluator import FredEvaluator
from common.timeseries import TimeseriesBundle

class FredStatistics:
    def __init__(self, **args):
        self.training, self.target = FredDataset(FredMeta.dataset_path).standard_split()
        self.evaluator = FredEvaluator(self.target, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))

# M4
from resources.m4.dataset import M4Dataset, M4Meta
from resources.m4.evaluator import M4Evaluator

class M4Statistics:
    def __init__(self, **args):
        self.training, self.target = M4Dataset(M4Meta.dataset_path).standard_split()
        self.evaluator = M4Evaluator(self.target, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))
    
# M3
from resources.m3.dataset import M3Dataset, M3Meta
from resources.m3.evaluator import M3Evaluator

class M3Statistics:
    def __init__(self, **args):
        self.training, self.target = M3Dataset(M3Meta.dataset_path).standard_split()
        self.evaluator = M3Evaluator(self.target, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))

    
# TOURISM
from resources.tourism.dataset import TourismDataset, TourismMeta
from resources.tourism.evaluator import TourismEvaluator

class TourismStatistics:
    def __init__(self, **args):
        self.training, self.target = TourismDataset(TourismMeta.dataset_path).standard_split()
        self.evaluator = TourismEvaluator(self.target, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))
    
# ELECTRICITY
from resources.electricity.dataset import ElectricityDataset, ElectricityMeta
from resources.electricity.evaluator import ElectricityEvaluator

class ElectricityStatisticsDeepAR:
    def __init__(self, **args):
        self.training, self.target = ElectricityDataset(ElectricityMeta.dataset_path).load_cache().split(
            lambda ts: ts.split_by_time(ElectricityMeta.deepar_split))
        self.target, _ = self.target.split(lambda ts: ts.split(24 * 7))
        self.evaluator = ElectricityEvaluator(self.target, precision=5, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))
    
class ElectricityStatisticsDeepFact:
    def __init__(self, **args):
        self.training, self.target = ElectricityDataset(ElectricityMeta.dataset_path).load_cache().split(
            lambda ts: ts.split_by_time(ElectricityMeta.deepfact_split))
        self.target, _ = self.target.split(lambda ts: ts.split(24 * 7))
        self.evaluator = ElectricityEvaluator(self.target, precision=5, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))
    
class ElectricityStatisticsLastWindow:
    def __init__(self, **args):
        self.training, self.target = ElectricityDataset(ElectricityMeta.dataset_path).standard_split()
        self.evaluator = ElectricityEvaluator(self.target, precision=5, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))
    
# TRAFFIC
from resources.traffic.dataset import TrafficDataset, TrafficMeta
from resources.traffic.evaluator import TrafficEvaluator

class TrafficStatisticsDeepAR:
    def __init__(self, **args):
        self.training, self.target = TrafficDataset(TrafficMeta.dataset_path).load_cache().split(
            lambda ts: ts.split_by_time(TrafficMeta.deepar_split))
        self.target, _ = self.target.split(lambda ts: ts.split(24 * 7))
        self.evaluator = TrafficEvaluator(self.target, precision=5, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))

class TrafficStatisticsDeepFact:
    def __init__(self, **args):
        self.training, self.target = TrafficDataset(TrafficMeta.dataset_path).load_cache().split(
            lambda ts: ts.split_by_time(TrafficMeta.deepfact_split))
        self.target, _ = self.target.split(lambda ts: ts.split(24 * 7))
        self.evaluator = TrafficEvaluator(self.target, precision=5, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))
    
class TrafficStatisticsLastWindow:
    def __init__(self, **args):
        self.training, self.target = TrafficDataset(TrafficMeta.dataset_path).standard_split()
        self.evaluator = TrafficEvaluator(self.target, precision=5, **args)
        
    def evaluate(self, predictions):
        return self.evaluator.evaluate(self.training.future_values([p[~np.isnan(p)] for p in predictions]))
    
def collect_statistics(filter_path, evaluator):
    statistics = EnsembleStatistics(filter_path=filter_path, evaluator=evaluator)

    bootstrap = statistics.bootstrap(ensemble_keys=['repeats', 'lookback_period', 'loss_name'], 
                                     bootstrap_key='repeats', 
                                     bootstrap_size=5, 
                                     number_of_samples=1)

    return bootstrap

#######################################

def assemble_results(experiment_path):
#     fred_bootstrap = collect_statistics(f'{experiment_path}/fred.csv', FredStatistics())
#     m4_bootstrap = collect_statistics(f'{experiment_path}/M4.csv', M4Statistics())
#     m3_bootstrap = collect_statistics(f'{experiment_path}/M3.csv', M3Statistics())
#     tourism_bootstrap = collect_statistics(f'{experiment_path}/tourism.csv', TourismStatistics())
    electricity_deepar_bootstrap = collect_statistics(f'{experiment_path}/electricity_deepar.csv', ElectricityStatisticsDeepAR())
    electricity_deepfact_bootstrap = collect_statistics(f'{experiment_path}/electricity_deepfactors.csv', ElectricityStatisticsDeepFact())
    electricity_lw_bootstrap = collect_statistics(f'{experiment_path}/electricity_last_window.csv', ElectricityStatisticsLastWindow())
    traffic_deepar_bootstrap = collect_statistics(f'{experiment_path}/traffic_deepar.csv', TrafficStatisticsDeepAR())
    traffic_deepfact_bootstrap = collect_statistics(f'{experiment_path}/traffic_deepfactors.csv', TrafficStatisticsDeepFact())
    traffic_lw_bootstrap = collect_statistics(f'{experiment_path}/traffic_last_window.csv', TrafficStatisticsLastWindow())
    
    result = {
#         'fred': fred_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose()[['Yearly', 'Quarterly', 'Monthly', 'Weekly', 'Daily', 'Average']],
#         'm4': m4_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose()[['Yearly', 'Quarterly', 'Monthly', 'Others', 'Average']],
#         'm3': m3_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose()[['M3Year', 'M3Quart', 'M3Month', 'M3Other', 'Average']],
#         'tourism': tourism_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose()[['Yearly', 'Quarterly', 'Monthly', 'Average']],
        'electricity_deepar': electricity_deepar_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose(),
        'electricity_deepfact': electricity_deepfact_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose(),
        'electricity_lw': electricity_lw_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose(),
        'traffic_deepar': traffic_deepar_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose(),
        'traffic_deepfact': traffic_deepfact_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose(),
        'traffic_lw': traffic_lw_bootstrap.groupby('evaluation_key').mean()[['metric']].transpose()
    }
    
    for k, v in result.items():
        print(k)
        display(v)

In [5]:
assemble_results('/project/experiments/nbeats_meta/shared/*source_dataset=M4')

FileNotFoundError: [Errno 2] No such file or directory: 'project/storage/resources/m3/cache.dill'

In [4]:
assemble_results('/project/experiments/nbeats_meta/not_shared/*source_dataset=M4')

100%|██████████| 1/1 [00:00<00:00,  1.91it/s]

fred


evaluation_key,Yearly,Quarterly,Monthly,Weekly,Daily,Average
metric,15.061,13.481,6.245,5.709,9.21,11.7


m4


evaluation_key,Yearly,Quarterly,Monthly,Others,Average
metric,13.146,9.55,12.441,3.523,11.464


...

evaluation_key,Yearly,Quarterly,Monthly,Average
metric,24.05,14.78,19.32,18.92


electricity


evaluation_key,metric
metric,0.10097


traffic


evaluation_key,metric
metric,0.15158


In [5]:
assemble_results('/project/experiments/nbeats_meta/shared/*source_dataset=FRED')

100%|██████████| 1/1 [00:00<00:00,  2.05it/s]

fred


evaluation_key,Yearly,Quarterly,Monthly,Weekly,Daily,Average
metric,15.344,13.334,6.024,5.187,8.634,11.724


m4


evaluation_key,Yearly,Quarterly,Monthly,Others,Average
metric,13.267,9.634,12.694,4.892,11.701


...

evaluation_key,Yearly,Quarterly,Monthly,Average
metric,23.53,14.47,21.23,19.94


electricity


evaluation_key,metric
metric,0.09209


traffic


evaluation_key,metric
metric,0.25842


In [6]:
assemble_results('/project/experiments/nbeats_meta/not_shared/*source_dataset=FRED')

100%|██████████| 1/1 [00:00<00:00,  2.18it/s]

fred


evaluation_key,Yearly,Quarterly,Monthly,Weekly,Daily,Average
metric,15.417,13.341,6.031,5.177,8.603,11.761


m4


evaluation_key,Yearly,Quarterly,Monthly,Others,Average
metric,13.272,9.596,12.676,4.696,11.675


...

evaluation_key,Yearly,Quarterly,Monthly,Average
metric,23.43,14.45,20.47,19.46


electricity


evaluation_key,metric
metric,0.08615


traffic


evaluation_key,metric
metric,0.25986


In [23]:
assemble_results('/project/experiments/nbeats_meta/shared/*source_dataset=M4')

100%|██████████| 1/1 [00:00<00:00,  2.12it/s]

electricity_deepar


evaluation_key,metric
metric,0.09408


electricity_deepfact


evaluation_key,metric
metric,0.09221


...

evaluation_key,metric
metric,0.14721


traffic_deepfact


evaluation_key,metric
metric,0.245


traffic_lw


evaluation_key,metric
metric,0.15569


In [24]:
assemble_results('/project/experiments/nbeats_meta/not_shared/*source_dataset=M4')

100%|██████████| 1/1 [00:00<00:00,  2.13it/s]

electricity_deepar


evaluation_key,metric
metric,0.10206


electricity_deepfact


evaluation_key,metric
metric,0.09535


...

evaluation_key,metric
metric,0.15175


traffic_deepfact


evaluation_key,metric
metric,0.25021


traffic_lw


evaluation_key,metric
metric,0.15967


In [25]:
assemble_results('/project/experiments/nbeats_meta/shared/*source_dataset=FRED')

100%|██████████| 1/1 [00:00<00:00,  1.92it/s]

electricity_deepar


evaluation_key,metric
metric,0.09136


electricity_deepfact


evaluation_key,metric
metric,0.08379


...

evaluation_key,metric
metric,0.26001


traffic_deepfact


evaluation_key,metric
metric,0.35461


traffic_lw


evaluation_key,metric
metric,0.2651


In [26]:
assemble_results('/project/experiments/nbeats_meta/not_shared/*source_dataset=FRED')

100%|██████████| 1/1 [00:00<00:00,  2.15it/s]

electricity_deepar


evaluation_key,metric
metric,0.08583


electricity_deepfact


evaluation_key,metric
metric,0.08003


...

evaluation_key,metric
metric,0.25867


traffic_deepfact


evaluation_key,metric
metric,0.34753


traffic_lw


evaluation_key,metric
metric,0.2652
